### Import modules

In [37]:
# Imports
import os
import cv2 as cv
import numpy as np
from tkinter import Tk
from tkinter.filedialog import askdirectory
from functions import *

### Run code on 2 sample images

In [38]:
# Image Paths
img0_path = r"C:\Users\Vraj\Documents\Thesis\Image-Algorithm\Sample Images\Image_Set_1\OCT Images\OCT_p7.png"
img1_path = r"C:\Users\Vraj\Documents\Thesis\Image-Algorithm\Sample Images\Image_Set_1\OCT Images\OCT_p6.png"
# img0_path = r"C:\Users\Vraj\Documents\Thesis\Image-Algorithm\Sample Images\Test_Images\1.png"
# img1_path = r"C:\Users\Vraj\Documents\Thesis\Image-Algorithm\Sample Images\Test_Images\2.png"


# Load Images
img0 = cv.imread(img0_path)
img1 = cv.imread(img1_path)

# Convert images to grayscale
img0_gray = cv.cvtColor(img0, cv.COLOR_RGB2GRAY)
img1_gray = cv.cvtColor(img1, cv.COLOR_RGB2GRAY)

# Extract features
features0 = featureExtraction(img0_gray)
features1 = featureExtraction(img1_gray)

# Match features
matches = featureMatching(features0, features1)

# Calculate homography
homography, _ = cv.findHomography(
    features0.matched_pts, features1.matched_pts, cv.RANSAC, 5.0
)

# Calculate dimensions of image to be warped
height, width, _ = img1.shape

# Warp target image to source image using homography matrix
warped = cv.warpPerspective(
    img0,
    homography,
    (width, height),
    borderMode=cv.BORDER_CONSTANT,  # need to see if applying a border around the image would be a good idea to avoid errors
    borderValue=(0, 0, 0, 0),
)

# Some stuff is happening here
output = np.zeros((height, width, 3), np.uint8)
alpha = warped[:, :, 2] / 255.0
output[:, :, 0] = (1.0 - alpha) * img1[:, :, 0] + alpha * warped[:, :, 0]
output[:, :, 1] = (1.0 - alpha) * img1[:, :, 1] + alpha * warped[:, :, 1]
output[:, :, 2] = (1.0 - alpha) * img1[:, :, 2] + alpha * warped[:, :, 2]

# Save image
cv.imwrite('output.png', output)


True

### Calculate normalised cross-corellation to assess registration performance

In [39]:
corellation = crossCorellation(img1, output)
print(corellation)

97.10%


### Calculate absolute difference between target and output images

In [40]:
absDifference(img1, output)